In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
import os
from torchvision import datasets, transforms
from torch.utils.data import Dataset, DataLoader, ConcatDataset, random_split
from PIL import Image

In [ ]:
num_classes = 2
learning_rate = 0.001
dropout_rate = 0.5
batch_size = 32
num_epochs = 25

In [ ]:
class ResNetIrisTumor(nn.Module):
    def __init__(self, num_classes, dropout_rate):
        super(ResNetIrisTumor, self).__init__()
        self.resnet = models.resnet18(pretrained=True)
        self.resnet.fc = nn.Sequential(
            nn.Dropout(dropout_rate),
            nn.Linear(self.resnet.fc.in_features, num_classes)
        )

    def forward(self, x):
        return self.resnet(x)

In [ ]:
model = ResNetIrisTumor(num_classes=num_classes, dropout_rate=dropout_rate)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 136MB/s]


In [ ]:
class CustomDataset(Dataset):
    def __init__(self, folder_path, label, transform=None):
        self.folder_path = folder_path
        self.label = label
        self.transform = transform
        self.image_paths = [os.path.join(folder_path, img) for img in os.listdir(folder_path)]

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        image = Image.open(img_path).convert('RGB')
        if self.transform:
            image = self.transform(image)
        return image, self.label

In [ ]:
tumor_path = "/content/drive/MyDrive/INFOSYS SPRINGBOARD/IRIS/YES"
no_tumor_path = "/content/drive/MyDrive/INFOSYS SPRINGBOARD/IRIS/NO"

In [ ]:
tumor_dataset = CustomDataset(tumor_path, label=1, transform=transform)
no_tumor_dataset = CustomDataset(no_tumor_path, label=0, transform=transform)
combined_dataset = ConcatDataset([tumor_dataset, no_tumor_dataset])
train_size = int(0.8 * len(combined_dataset))
val_size = len(combined_dataset) - train_size
train_data, val_data = random_split(combined_dataset, [train_size, val_size])

In [ ]:
batch_size = 32
train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_data, batch_size=batch_size, shuffle=False)
num_epochs=10

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    for inputs, labels in train_loader:
        inputs, labels = inputs.to('cpu'), labels.to('cpu')
        optimizer.zero_grad()

        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * inputs.size(0)
        _, predicted = outputs.max(1)
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()

    epoch_loss = running_loss / len(train_loader.dataset)
    epoch_acc = 100. * correct / total

    print(f'Epoch {epoch+1}/{num_epochs}, Loss: {epoch_loss:.4f}, Accuracy: {epoch_acc:.2f}%')

    model.eval()
    val_loss = 0.0
    val_correct = 0
    val_total = 0
    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs, labels = inputs.to('cpu'), labels.to('cpu')

            outputs = model(inputs)
            loss = criterion(outputs, labels)

            val_loss += loss.item() * inputs.size(0)
            _, predicted = outputs.max(1)
            val_total += labels.size(0)
            val_correct += predicted.eq(labels).sum().item()

    val_epoch_loss = val_loss / len(val_loader.dataset)
    val_epoch_acc = 100. * val_correct / val_total
    print(f'Validation Loss: {val_epoch_loss:.4f}, Validation Accuracy: {val_epoch_acc:.2f}%')



Epoch 1/10, Loss: 0.3865, Accuracy: 83.51%
Validation Loss: 0.1499, Validation Accuracy: 93.62%
Epoch 2/10, Loss: 0.0867, Accuracy: 96.28%
Validation Loss: 0.9232, Validation Accuracy: 80.85%
Epoch 3/10, Loss: 0.0376, Accuracy: 97.87%
Validation Loss: 5.5579, Validation Accuracy: 65.96%
Epoch 4/10, Loss: 0.0512, Accuracy: 97.87%
Validation Loss: 0.1208, Validation Accuracy: 95.74%
Epoch 5/10, Loss: 0.0059, Accuracy: 100.00%
Validation Loss: 0.1230, Validation Accuracy: 95.74%
Epoch 6/10, Loss: 0.0047, Accuracy: 100.00%
Validation Loss: 0.0848, Validation Accuracy: 95.74%
Epoch 7/10, Loss: 0.0125, Accuracy: 99.47%
Validation Loss: 0.1772, Validation Accuracy: 93.62%
Epoch 8/10, Loss: 0.0019, Accuracy: 100.00%
Validation Loss: 0.1832, Validation Accuracy: 93.62%
Epoch 9/10, Loss: 0.0006, Accuracy: 100.00%
Validation Loss: 0.1858, Validation Accuracy: 93.62%
Epoch 10/10, Loss: 0.0003, Accuracy: 100.00%
Validation Loss: 0.1786, Validation Accuracy: 93.62%


In [ ]:
torch.save(model.state_dict(), '/content/drive/MyDrive/INFOSYS SPRINGBOARD/Resnet.pth')


In [ ]:
!pip install onnx

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.0/16.0 MB 75.6 MB/s eta 0:00:00


In [ ]:
import torch

dummy_input = torch.randn(1, 3, 224, 224)  # Adjust dimensions as needed
torch.onnx.export(model, dummy_input, '/content/drive/MyDrive/INFOSYS SPRINGBOARD/Resnet.onnx')


In [ ]:
scripted_model = torch.jit.script(model)  # Or torch.jit.trace for tracing
scripted_model.save('/content/drive/MyDrive/INFOSYS SPRINGBOARD/Resnet.pt')


In [ ]:
import torch
from torch.quantization import quantize_dynamic

# Apply dynamic quantization
quantized_model = quantize_dynamic(model, {torch.nn.Linear}, dtype=torch.qint8)
torch.save(quantized_model.state_dict(), '/content/drive/MyDrive/INFOSYS SPRINGBOARD/Resnet_quantized.pth')


In [ ]:
import torch.nn.utils.prune as prune

# Apply pruning to layers
# Changed model.fc to model.resnet.fc to access the fully connected layer
prune.random_unstructured(model.resnet.fc[1], name="weight", amount=0.5)  # Example: prune 50% weights of the linear layer within the sequential
torch.save(model.state_dict(), '/content/drive/MyDrive/INFOSYS SPRINGBOARD/Resnet_pruned.pth')

In [ ]:
torch.save(model.state_dict(), '/content/drive/MyDrive/INFOSYS SPRINGBOARD/Resnet_weights_only.pth')


In [ ]:
# For ONNX
torch.onnx.export(model, dummy_input, '/content/drive/MyDrive/INFOSYS SPRINGBOARD/Resnet_optimized.onnx')

# For TorchScript
scripted_model = torch.jit.script(model)
scripted_model.save('/content/drive/MyDrive/INFOSYS SPRINGBOARD/Resnet_optimized.pt')
